In [1]:
#结合focal loss 函数讲解

In [61]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


print(tf.__version__)
print(np.__version__)

mnist = np.load("mnist.npz")
x_train, y_train, x_test, y_test = mnist['x_train'],mnist['y_train'],mnist['x_test'],mnist['y_test']

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = np.int32(y_train)
y_test = np.int32(y_test)
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
y_train = tf.one_hot(y_train,depth=10)
y_test = tf.one_hot(y_test,depth=10)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(100).batch(32)


2.0.0
1.19.1


In [62]:
x_test.shape

(10000, 28, 28, 1)

In [63]:
def MyModel():
    inputs = tf.keras.Input(shape=(28,28,1), name='digits')
    x = tf.keras.layers.Conv2D(32, 3, activation='relu')(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(10,activation='softmax', name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


In [64]:
# #多分类的focal loss 损失函数
# class FocalLoss(tf.keras.losses.Loss):

#     def __init__(self,gamma=2.0,alpha=0.25):
#         self.gamma = gamma
#         self.alpha = alpha
#         super(FocalLoss, self).__init__()

#     def call(self,y_true,y_pred):
#         y_pred = tf.nn.softmax(y_pred,axis=-1)
#         epsilon = tf.keras.backend.epsilon()
#         y_pred = tf.clip_by_value(y_pred, epsilon, 1.0)
        
       
#         y_true = tf.cast(y_true,tf.float32)
        
#         loss = -  y_true * tf.math.pow(1 - y_pred, self.gamma) * tf.math.log(y_pred)
        
#         loss = tf.math.reduce_sum(loss,axis=1)
#         return loss
    

In [65]:
def FocalLoss(gamma=2.0,alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = tf.nn.softmax(y_pred,axis=-1)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0)

        y_true = tf.cast(y_true,tf.float32)

        loss = -  y_true * tf.math.pow(1 - y_pred, gamma) * tf.math.log(y_pred)

        loss = tf.math.reduce_sum(loss,axis=1)
        return  loss
    return focal_loss_fixed

In [102]:
class LearningRateMonitor(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        if batch % 100 == 0:
            print("...Training: start of batch {}; got learning rate: {}".format(
                batch, tf.keras.backend.get_value(self.model.optimizer._decayed_lr(tf.float32))))



In [103]:

model = MyModel()
model.compile(optimizer = tf.keras.optimizers.Adam(0.001), #优化器
              loss =  FocalLoss(gamma=2.0,alpha=0.25), #损失函数
              metrics = [tf.keras.metrics.CategoricalAccuracy()]
             ) #评估函数


In [104]:
model.fit(train_ds, epochs=5, validation_data=test_ds, callbacks=[LearningRateMonitor()])

Epoch 1/5
...Training: start of batch 0; got learning rate: 0.0010000000474974513
    100/Unknown - 8s 75ms/step - loss: 1.1538 - categorical_accuracy: 0.7806...Training: start of batch 100; got learning rate: 0.0010000000474974513
    200/Unknown - 14s 71ms/step - loss: 1.0860 - categorical_accuracy: 0.8261...Training: start of batch 200; got learning rate: 0.0010000000474974513
    300/Unknown - 21s 70ms/step - loss: 1.0510 - categorical_accuracy: 0.8528...Training: start of batch 300; got learning rate: 0.0010000000474974513
    400/Unknown - 28s 70ms/step - loss: 1.0273 - categorical_accuracy: 0.8713...Training: start of batch 400; got learning rate: 0.0010000000474974513
    500/Unknown - 38s 75ms/step - loss: 1.0136 - categorical_accuracy: 0.8816...Training: start of batch 500; got learning rate: 0.0010000000474974513
    600/Unknown - 47s 78ms/step - loss: 1.0005 - categorical_accuracy: 0.8914...Training: start of batch 600; got learning rate: 0.0010000000474974513
    700/Unkno

1875/1875 [==============================] - 147s 78ms/step - loss: 0.8843 - categorical_accuracy: 0.9821 - val_loss: 0.8872 - val_categorical_accuracy: 0.9798
Epoch 4/5
...Training: start of batch 0; got learning rate: 0.0010000000474974513
1875/1875 [==============================] - 131s 70ms/step - loss: 0.8805 - categorical_accuracy: 0.9852 - val_loss: 0.8862 - val_categorical_accuracy: 0.9798
Epoch 5/5
...Training: start of batch 0; got learning rate: 0.0010000000474974513
1800/1875 [===========================>..] - ETA: 5s - loss: 0.8775 - categorical_accuracy: 0.9873...Training: start of batch 1800; got learning rate: 0.0010000000474974513


1875/1875 [==============================] - 141s 75ms/step - loss: 0.8775 - categorical_accuracy: 0.9874 - val_loss: 0.8859 - val_categorical_accuracy: 0.9804
